# Seasonal Futures - Calendar Windows

In [ ]:
from __future__ import annotations

import pandas as pd

from seasonal_analysis.constants import (
    DEFAULT_SYMBOL,
    DEFAULT_START_DATE,
    DEFAULT_END_DATE,
    LOOKBACK_YEARS,
    MONTHS,
    ENTRY_DAY_RANGE,
    EXIT_DAY_RANGE,
    MIN_LEN_DAYS,
    MAX_LEN_DAYS,
    DIRECTION,
    SMOOTH,
    EXCLUDE_INCOMPLETE_LAST_YEAR,
)
from seasonal_analysis.load_data import load_price_data
from seasonal_analysis.analysis import (
    run_seasonal_analysis,
)
from seasonal_analysis.plotting import (
    plot_seasonal_curve_with_windows,
    plot_per_year_pnl,
    plot_seasonal_stacks_by_lookback,
)

# Core selector (override here for a different futures contract)
symbol = DEFAULT_SYMBOL  # e.g., "ES", "CL", "NG", etc.
start_date = DEFAULT_START_DATE
end_date = DEFAULT_END_DATE

In [ ]:
# Load daily futures data
fut_df: pd.DataFrame = load_price_data(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    granularity="D",
)

In [ ]:
# Run generic seasonal analysis (asset-aware via symbol)
res = run_seasonal_analysis(
    symbol=symbol,
    df=fut_df,
    start=start_date,
    end=end_date,
    lookback_years=LOOKBACK_YEARS,
    months=MONTHS,
    entry_day_range=ENTRY_DAY_RANGE,
    exit_day_range=EXIT_DAY_RANGE,
    min_len_days=MIN_LEN_DAYS,
    max_len_days=MAX_LEN_DAYS,
    direction=DIRECTION,
    min_trades=10,
    min_win_rate=0.80,
    smooth=SMOOTH,
    exclude_incomplete_last_year=EXCLUDE_INCOMPLETE_LAST_YEAR,
)

In [ ]:
print(f"Symbol: {res.symbol}")
print(f"Years available: {res.years_available}")
print("Best windows:")
for w in res.top_windows[:10]:
    print(w)

In [ ]:
# 1) Shade top windows over the seasonal curve
if res.top_windows:
    plot_seasonal_curve_with_windows(
        res.seasonal_curve,
        res.top_windows[:5],
        title=f"{res.symbol} seasonal curve + top windows",
    )

    # 2) Bar chart for one specific window
    w0 = res.top_windows[0]
    plot_per_year_pnl(res.per_year_results, w0.entry_mmdd, w0.exit_mmdd)

In [ ]:
# 3) Plot seasonal stacks for multiple lookbacks on the same futures contract
plot_seasonal_stacks_by_lookback(
    fut_df,
    lookbacks=(5, 10, LOOKBACK_YEARS),
    title=f"{res.symbol} seasonal closes (5/10/{LOOKBACK_YEARS}y)",
)